In [56]:
import numpy as np 
import pandas as pd

n = 250

Y1 = np.random.normal(loc=10, scale=2, size=n)
Y2 = np.random.normal(loc=14, scale=1.4, size=n)

ids = np.arange(1, n + 1)

df = pd.DataFrame({
    "ID": ids,
    "Y1": Y1,
    "Y2": Y2
})

#Y1 = population mean 10, S.D 2, Var 4
#Y2 = population mean 14, S.D 1.4, Var 1.96

In [57]:
df["diff"] = df["Y2"] - df["Y1"]

#diff s normally distributed based on generated data.
#Global parameters
#E(diff) = E(Y2) - E(Y1) = 14-10 = 4
#Var(diff) = Var(Y2) + Var(Y1) = 1.96+4 = 5.96
#SD(diff) = sqrt(5.96) = 2.44

#However the underlying distribution of Y1, Y2 and diff is unknown (according to problem statement)
#Null Hypotheis: The values for diff came from a normal distribution
#We use Shapiro–Wilk test to test the Null Hypothesis


In [58]:
# Descriptive statistics
def desc_stats(data):
    mean = np.mean(data)
    std = np.std(data, ddof=1)
    se = std / np.sqrt(len(data))
    ci_lower = mean - 1.96 * se
    ci_upper = mean + 1.96 * se
    return mean, std, se, ci_lower, ci_upper

In [59]:
stats_y1 = desc_stats(df["Y1"])
stats_y2 = desc_stats(df["Y2"])
stats_diff = desc_stats(df["diff"])

print("RESULTS SUMMARY")
print("="*60)
print(f"\nY1 Statistics:")
print(f"  Mean: {stats_y1[0]:.4f}, SD: {stats_y1[1]:.4f}")
print(f"  95% CI: [{stats_y1[3]:.4f}, {stats_y1[4]:.4f}]")

print(f"\nY2 Statistics:")
print(f"  Mean: {stats_y2[0]:.4f}, SD: {stats_y2[1]:.4f}")
print(f"  95% CI: [{stats_y2[3]:.4f}, {stats_y2[4]:.4f}]")

print(f"\nPaired Differences (Y2 - Y1):")
print(f"  Mean: {stats_diff[0]:.4f}, SD: {stats_diff[1]:.4f}")
print(f"  95% CI: [{stats_diff[3]:.4f}, {stats_diff[4]:.4f}]")

RESULTS SUMMARY

Y1 Statistics:
  Mean: 9.9333, SD: 2.1106
  95% CI: [9.6717, 10.1949]

Y2 Statistics:
  Mean: 13.9565, SD: 1.2903
  95% CI: [13.7966, 14.1165]

Paired Differences (Y2 - Y1):
  Mean: 4.0232, SD: 2.5107
  95% CI: [3.7120, 4.3345]


In [60]:
#Generate order statistics for standard normal distribution
#Any normal distribution can be converted to standard normal distribution
#The same order statistics (the function in this case) can be used for other examples
def generate_order_statistics(n):
    iterations=20000
    m = np.zeros(n)
    for r in range(iterations):
        s = np.random.normal(0, 1, n)
        m += np.sort(s)
    return m / iterations

In [61]:
#Generate covariance matrix for order statistics for standard normal distribution
#Any normal distribution can be converted to standard normal distribution
#The same order statistics (the function in this case) can be used for other examples
def generate_order_covariance(n):
    iterations=20000

    sum_vec = np.zeros(n)
    sum_outer = np.zeros((n, n))

    for r in range(iterations):
        s = np.random.normal(0, 1, n)
        ordered = np.sort(s)

        sum_vec += ordered
        sum_outer += np.outer(ordered, ordered)

    mean_vec = sum_vec / iterations

    covariance_matrix = (sum_outer / iterations) - np.outer(mean_vec, mean_vec)

    return covariance_matrix

In [62]:
def Shapiro_Wilk(vector_sample, m, sigma):

    x = np.asarray(vector_sample)
    x_ordered = np.sort(x)

    x_bar = np.mean(x)

    denominator = np.sum((x - x_bar) ** 2)

    a_num = np.linalg.solve(sigma, m)
    a = a_num / np.sqrt(a_num @ m)

    numerator = (np.sum(a * x_ordered)) ** 2

    W = numerator / denominator

    return W


In [63]:
n = len(df["diff"])
m = generate_order_statistics(n)
sigma = generate_order_covariance(n)

W = Shapiro_Wilk(df["diff"], m, sigma)

print(f"W = {W}")


W = 2.0028125478949934


In [64]:
from scipy.stats import shapiro
W2, p = shapiro(df["diff"])

print(f"W = {W2}, p = {p}")

W = 0.9960297526292927, p = 0.7776017639062172
